In [1]:
# import libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns 
import html
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings; warnings.simplefilter('ignore')
from VE_scraper_functions import *
from chromedriver_py import binary_path # this will get you the path variable
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniel.Lang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
path = "park_scrape_content_dataset.xlsx"
park_scrape_dataset2 = pd.read_excel(path)
park_scrape_dataset2 = park_scrape_dataset2[park_scrape_dataset2['content'] != "page doesn't exist"]
park_scrape_dataset2.head

<bound method NDFrame.head of        Unnamed: 0  index                                       website page                                            content website location  park
0               0      0  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI
1               1      1  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI
2               2      2  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI
3               3      3  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI
4               4      4  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI
...           ...    ...                                              

In [3]:
website_stems = park_scrape_dataset2[['website page', 'content', 'park']]
website_stems.head

<bound method NDFrame.head of                                             website page                                            content  park
0      https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  ABLI
1      https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  ABLI
2      https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  ABLI
3      https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  ABLI
4      https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...  ABLI
...                                                  ...                                                ...   ...
14663  https://home.nps.gov/zion/planyourvisit/zion-c...  Zion Canyon Shuttle System - Zion National Par...  ZION
14664  https://home.nps.gov/zion/planyourvisit/kolob-...  

In [4]:
def extract_text(url):
    start = url.find('planyourvisit/') + len('planyourvisit/')
    end = url.find('.htm', start)
    if start != -1 and end != -1:
        return url[start:end]
    return None

park_scrape_dataset2['url_stem'] = park_scrape_dataset2['website page'].apply(extract_text)
park_scrape_dataset2.head

<bound method NDFrame.head of        Unnamed: 0  index                                       website page                                            content website location  park                      url_stem
0               0      0  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI                    directions
1               1      1  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI                    directions
2               2      2  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI                    directions
3               3      3  https://home.nps.gov/abli/planyourvisit/direct...  Directions - Abraham Lincoln Birthplace Nation...              NaN  ABLI                    directions
4               4      4  https://home.nps.gov/abli/planyourvisit/dire

In [5]:
value_counts = park_scrape_dataset2['url_stem'].value_counts()
most_common = value_counts[value_counts > 100]
print(most_common)

directions                2915
index                      763
trip-ideas                 423
passes                     420
event-search               418
conditions                 409
maps                       408
basicinfo                  401
calendar                   401
fees                       385
accessibility              342
safety                     339
things2do                  331
hours                      327
pets                       235
permitsandreservations     219
weather                    183
nearbyattractions          178
placestogo                 161
eatingsleeping             144
Name: url_stem, dtype: int64


In [6]:
# Define accessibility words
accessibility_words = [
    "wheelchair", "disability", "impaired", "disabilities",
    "handicap", " ada ", "braille", "disabled", "handicapped", "access", "accessibility"
]

# Function to count accessibility words in text
def count_accessibility_words(text):
    word_counts = {word: text.lower().count(word) for word in accessibility_words}
    return word_counts

# Apply the function to 'url_stem' column and convert to DataFrame
word_counts_df = park_scrape_dataset2['url_stem'].apply(count_accessibility_words).apply(pd.Series)
word_counts_df['access_page'] = word_counts_df.apply(lambda row: 1 if row.any() >= 1 else 0, axis=1)
word_counts_df = word_counts_df[['access_page']]

# Display the resulting DataFrame
print(word_counts_df.head())

   access_page
0            0
1            0
2            0
3            0
4            0


In [7]:
park_scrape_dataset2 = pd.concat([park_scrape_dataset2, word_counts_df], axis=1)
park_scrape_dataset2 = park_scrape_dataset2.drop("content", axis="columns")
park_scrape_dataset2.head

<bound method NDFrame.head of        Unnamed: 0  index                                       website page website location  park                      url_stem  access_page
0               0      0  https://home.nps.gov/abli/planyourvisit/direct...              NaN  ABLI                    directions            0
1               1      1  https://home.nps.gov/abli/planyourvisit/direct...              NaN  ABLI                    directions            0
2               2      2  https://home.nps.gov/abli/planyourvisit/direct...              NaN  ABLI                    directions            0
3               3      3  https://home.nps.gov/abli/planyourvisit/direct...              NaN  ABLI                    directions            0
4               4      4  https://home.nps.gov/abli/planyourvisit/direct...              NaN  ABLI                    directions            0
...           ...    ...                                                ...              ...   ...                    

In [8]:
park_word_counts = park_scrape_dataset2.groupby('park')['access_page'].sum().reset_index()
# Display the resulting DataFrame
print(park_word_counts.head())

   park  access_page
0  ABLI            0
1  ACAD            1
2  ADAM            1
3  AFBG            1
4  AGFO            1


In [9]:
park_word_counts_zero = park_word_counts[park_word_counts['access_page'] == 0]
print(park_word_counts_zero)

     park  access_page
0    ABLI            0
10   AMME            0
17   APPA            0
18   ARCH            0
20   ARPO            0
..    ...          ...
399  WHHO            0
402  WHSA            0
408  WOTR            0
415  YOSE            0
418  ZION            0

[76 rows x 2 columns]
